# TLS & Certificates: Interactive Debugging and Common Pitfalls
https://github.com/HG00/talk-tls-therapy

## Section 1: Basic Certificate Inspection

In [ ]:

import ssl
import socket
from datetime import datetime, timezone

def get_cert_info(hostname, port=443):
    context = ssl.create_default_context()
    with socket.create_connection((hostname, port)) as sock:
        with context.wrap_socket(sock, server_hostname=hostname) as ssock:
            cert = ssock.getpeercert()
            return cert

cert = get_cert_info('example.com')
print("Issuer:", cert['issuer'])
print("Subject:", cert['subject'])
print("Validity:", cert['notBefore'], "to", cert['notAfter'])


## Section 2: Certificate Date Validation

In [ ]:

def is_cert_valid_now(cert):
    date_format = "%b %d %H:%M:%S %Y %Z"
    not_before = datetime.strptime(cert['notBefore'], date_format).replace(tzinfo=timezone.utc)
    not_after = datetime.strptime(cert['notAfter'], date_format).replace(tzinfo=timezone.utc)
    now = datetime.now(timezone.utc)
    return not_before <= now <= not_after

print("Is certificate valid now?", is_cert_valid_now(cert))


## Section 3: Certificate Chain Validation

In [ ]:

import requests

def test_cert_chain(url):
    try:
        response = requests.get(url)
        print("Successful connection:", response.status_code)
    except requests.exceptions.SSLError as e:
        print("SSL error:", e)

test_cert_chain('https://expired.badssl.com')
test_cert_chain('https://wrong.host.badssl.com')


## Section 4: Using curl for TLS Debugging

In [ ]:
# Wrong date
!curl -v https://expired.badssl.com

In [ ]:
# Wrong CN or SAN
!curl -v https://wrong.host.badssl.com

In [ ]:
# Unknown Root CA
!curl -v https://untrusted-root.badssl.com

In [ ]:
# Bad chain
!curl -v https://invalid-expected-sct.badssl.com

In [ ]:
# Check your client TLS - just hit this URL for a json response
!curl https://check.tls.support/

In [ ]:
# Get some details on a certificate
!curl echo | openssl s_client -connect cloud.cambiumnetworks.com:443 | openssl x509 -noout -text

## Section 5: Detecting MITM Proxies


Detect MITM proxies using curl:

```
curl -v https://example.com
```

Review the issuer in the output for suspicious certificates.


## Section 6: When TLS Goes Wrong - Quick Reference

In [ ]:

issues_examples = {
    "Expired Certificate": "https://expired.badssl.com",
    "Wrong Hostname": "https://wrong.host.badssl.com",
    "Self-signed Certificate": "https://self-signed.badssl.com",
    "Untrusted Root": "https://untrusted-root.badssl.com",
    "Incomplete Chain": "https://incomplete-chain.badssl.com",
    "Invalid Intermediate": "https://invalid-expected-sct.badssl.com",
}

for issue, url in issues_examples.items():
    print(f"Testing {issue}:")
    test_cert_chain(url)
    print("-" * 60)


## Section 7: Hands-On Exercise - Debug Your Own TLS

In [ ]:

# Uncomment and run to debug your own host
# user_host = input("Enter your host to debug TLS: ")
# cert = get_cert_info(user_host)
# print(cert)
# test_cert_chain(f'https://{user_host}')
